In [ ]:
import json
import requests
base_path= 'http://127.0.0.1:5002/api/v1'

# Use this if you are testing against an instance with OPENID auth
# %run fetch_openid_bearer.ipynb
# print (auth_headers)

# Everything you need to create an assistant from scratch

## 1. Create Modules

In [ ]:
r = requests.post(f'{base_path}/module', json=
{
  "name": "GPT35 on Azure",
  "specId": "d7975f09-2b33-4ddf-bc29-cdc6c64a6781",
  "parameters": [
    {
      "name": "M_LLM_AZURE_OPENAI_API_KEY",
      "value": "*********"
    },{
      "name": "M_LLM_AZURE_OPENAI_API_VERSION",
      "value": "2023-07-01-preview"
    },{
      "name": "M_LLM_AZURE_OPENAI_DEPLOYMENT_NAME",
      "value": "aiphuahhailang01"
    },{
      "name": "M_LLM_AZURE_OPENAI_ENDPOINT",
      "value": "https://aiphuahhaiopai01.openai.azure.com"
    },{
      "name": "M_LLM_AZURE_OPENAI_MODEL_NAME",
      "value": "gpt-35-turbo"
    }
  ]
})

llm_id_gpt35 = r.json().get("id")
print("GPT35 ID: " + llm_id_gpt35)

In [ ]:
r = requests.post(f'{base_path}/module', json=
{
  "name": "GPT4o on Azure",
  "specId": "d7975f09-2b33-4ddf-bc29-cdc6c64a6781",
  "parameters": [
    {
      "name": "M_LLM_AZURE_OPENAI_API_KEY",
      "value": "*********"
    },{
      "name": "M_LLM_AZURE_OPENAI_API_VERSION",
      "value": "2023-07-01-preview"
    },{
      "name": "M_LLM_AZURE_OPENAI_DEPLOYMENT_NAME",
      "value": "aip4d-gpt4o"
    },{
      "name": "M_LLM_AZURE_OPENAI_ENDPOINT",
      "value": "https://aiphuahhaiopai04.openai.azure.com"
    },{
      "name": "M_LLM_AZURE_OPENAI_MODEL_NAME",
      "value": "gpt-4o"
    }
  ]
})

llm_id_gpt4o = r.json().get("id")
print("GPT4o ID: " + llm_id_gpt4o)

In [ ]:
r = requests.post(f'{base_path}/module', json=
{
  "name": "Azure hosted EMB - text ada-002",
  "specId": "846232c8-b6e9-4ee2-87ab-3e173768f06c",
  "parameters": [
    {
      "name": "M_EMB_AZURE_OPENAI_API_KEY",
      "value": "*********"
    },{
      "name": "M_EMB_AZURE_OPENAI_API_VERSION",
      "value": "2023-07-01-preview"
    },{
      "name": "M_EMB_AZURE_OPENAI_DEPLOYMENT_NAME",
      "value": "aiphuahhaiembb01"
    },{
      "name": "M_EMB_AZURE_OPENAI_ENDPOINT",
      "value": "https://aiphuahhaiopai01.openai.azure.com"
    },{
      "name": "M_EMB_AZURE_OPENAI_MODEL_NAME",
      "value": "text-embedding-ada-002"
    }
  ]
})

embedding_id = r.json().get("id")
print("Embedding model ID: " + embedding_id)

In [ ]:
r = requests.post(f'{base_path}/module', json=
{
  "name": "Local Vector Store Index",
  "specId": "2b929513-16fa-47a4-94b5-7036e16fa8dc",
  "parameters": []
})

vector_store_index_id = r.json().get("id")
print("Vector Store Index ID: " + vector_store_index_id)

In [ ]:
r = requests.post(f'{base_path}/module', json=
{
  "name": "Local Data File Storage",
  "specId": "b3a6b442-8123-49ee-b379-9ee1ff9099ae",
  "parameters": []
})

file_storage_id = r.json().get("id")
print("File Storage ID: " + file_storage_id)


## 2. Create a tag

In [ ]:
r = requests.post(f'{base_path}/tag', json={
  "name": "Tone Of Voice",
  "description": "A tag to filter for tone of voice specific resources in the application."
})
tag_id = r.json()['id']
print(f"Tag ID: {tag_id}")

## 3. Create DataSource

In [ ]:
r = requests.post(f'{base_path}/data-source', json={
  "name": "Tone Of Voice Documents",
  "description": "An index with all the documents related to tone of voice guidelines.",
  "tags": [{
    "id": tag_id
  }],
  "embeddingModelId": embedding_id,
  "vectorStoreId": vector_store_index_id,
  "fileStorageId": file_storage_id
})
data_source_id = r.json()['id']
print(f"Data Source ID: {data_source_id}")


## 4. Create Workspace

In [ ]:
r = requests.post(f'{base_path}/workspace', json={
  "name": "Home",
  "description": "My personal and private workspace.",
  "public": False # Default: False
})
workspace_id = r.json()['id']
print(f"Workspace ID: {workspace_id}")

## (optional) 5. Create System Instruction

In [ ]:
r = requests.post(f'{base_path}/system-instruction', json={
  "name": "Tone of voice assistant instruction",
  "tags": [{
    "id": tag_id
  }],
  "message": """
    You are a tone of voice expert who helps users craft emails and messages in alignment with the official Abu Dhabi government guidelines for customer experience document. 
    
    You will follow these steps:
    
    When a conversation starts, you ask for the original content of an email or message, which needs to be corrected into correct tone of voice. 
    
    Once you have the original content you translate it as per the tone of voice guidelines. 
    
    You are multilingual. If you are directed to speak in a certain language you will only speak in that language.  if you are spoken to in a certain language you will only reply in that language. The correction of the original message must be given in the same language as the original message was written in, unless otherwise specified by the user.
    
    Your response must be the corrected version of the provided content in the respective language, followed by a clear bullet point list on what has been corrected. Every list item must end with a reference to document that justifies the correction and which is following the syntax '<document_url>#<page number>', where <document_url> is either the absolute file path or an URL of the file (no document title), and where <page number> is the page number inside the document at which the relevant information is located. Do not correct things that you cannot justify like mentioned before. You are not allowed to skip the list, it must always be added. You are not allowed to add list items without citation. 
    
    Do not include visual markers for citations in the corrected response. 
    
    Review your own result and repeat your corrections until you cannot find any further rules to apply from the guideline documents before your return your final response. Include all corrections from each iteration in the final list of corrections and do not just change the text without a reference to the guideline you have been using.
    
    Only use documents from the context and never use external or publicly available documents.
    
    You may ask follow up questions.
    """
})
system_instruction_id = r.json()['id']
print(f"System Instruction ID: {system_instruction_id}")

## 6. Create Assistant

In [ ]:
r = requests.post(f'{base_path}/assistant', json={
  "name": "ToV Assistant",
  "description": "An assistant to support you in re-writing messages according to given guidelines and rules.",
  "tags": [{
    "id": tag_id
  }],
  "systemInstructionId": system_instruction_id,
  "customSystemInstruction": "At the end of a response, always ask a follow up question.",
  "greetingMessage": "Hello. I am helpful. Please ask me something.",
  "llmIds": [
    llm_id_gpt4o
  ],
  "dataSourceIds": [
    data_source_id
  ],
  "public": False # Default = False
})
assistant_id = r.json()['id']
print(f"Assistant ID: {assistant_id}")

## 7. Add assistant to workspace

In [ ]:
r = requests.post(f'{base_path}/workspace/{workspace_id}/assistants', json={
  "assistantIds": [
    assistant_id
  ]
})
workspace_assistant_relation_response = r.json()
print(f"Assistant added: {workspace_assistant_relation_response}")